# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [24]:
# !pip install langchain_chroma

In [27]:
# !pip install langchain_groq

In [30]:
# !pip install langchain-community

In [3]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 9.77MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [31]:
from langchain_chroma import Chroma
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import markdown
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [4]:
import pandas as pd

df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [60]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [61]:
# loop through the dataset and generate markdown content
for i in range(0, 104):

    title = df['المخالفة'].iloc[i]
    content = df['الغرامة'].iloc[i]

    markdown_content = f"# {title}\n\n"
    markdown_content += f"{content}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [62]:
#read each file
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

In [63]:
markdown_texts[1]

'<h1>قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.</h1>\n<p>الغرامة المالية 100 - 150 ريال</p>'

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [20]:
#!pip install langchain

In [64]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

In [65]:
documents[1]

Document(metadata={}, page_content='<h1>قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.</h1>\n<p>الغرامة المالية 100 - 150 ريال</p>')

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [33]:
%pip install -qU langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 10.0 MB/s eta 0:00:00


In [66]:
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [68]:
PROMPT_TEMPLATE="""
جب على السؤال اعتمادا على التالي فقط باللغة العربية :
السياق: {السياق}
السؤال: {السؤال}
 اجابتك:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["السياق", "السؤال"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [86]:
groq_api_key = "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"
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [87]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [88]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"السياق": context_text, "السؤال": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

# Answer with temprature = 0

In [89]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

Prompt after formatting:

جب على السؤال اعتمادا على التالي فقط باللغة العربية : 
السياق: <h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>

<h1>المراوغة بسرعة بين المركبات على الطرق العامة.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>
السؤال: ماهي الغرامة على القيادة بدون رخصة؟
 اجابتك:


> Finished chain.


{'السياق': '<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>\n\n<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>\n\n<h1>الغرامة المالية 3000 - 6000 ريال</h1>\n<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>\n\n<h1>المراوغة بسرعة بين المركبات على الطرق العامة.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 'السؤال': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'لا يوجد سؤال عن القيادة بدون رخصة في النص الم 제공. النص يحتوي على سلوكيات مخالفة على الطرق العامة مع الغرامات المالية المحددة، ولكن لا يوجد سؤال عن القيادة بدون رخصة.'}

In [90]:
print(f'السياق:\n{response["السياق"]}\n\nالسؤال:\n{response["السؤال"]}\n\nالاجابة: \n{response["text"]}')

السياق:
<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>

<h1>المراوغة بسرعة بين المركبات على الطرق العامة.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

السؤال:
ماهي الغرامة على القيادة بدون رخصة؟

الاجابة: 
لا يوجد سؤال عن القيادة بدون رخصة في النص الم 제공. النص يحتوي على سلوكيات مخالفة على الطرق العامة مع الغرامات المالية المحددة، ولكن لا يوجد سؤال عن القيادة بدون رخصة.


# Answer with tempreature = 0.4

In [84]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

Prompt after formatting:

جب على السؤال اعتمادا على التالي فقط باللغة العربية : 
السياق: <h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>

<h1>المراوغة بسرعة بين المركبات على الطرق العامة.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>
السؤال: ماهي الغرامة على القيادة بدون رخصة؟
 اجابتك:


> Finished chain.


{'السياق': '<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>\n\n<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>\n\n<h1>الغرامة المالية 3000 - 6000 ريال</h1>\n<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>\n\n<h1>المراوغة بسرعة بين المركبات على الطرق العامة.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 'السؤال': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'لا توجد غرامة مالية مذكورة على القيادة بدون رخصة في النص الم 제공.'}

In [85]:
print(f'السياق:\n{response["السياق"]}\n\nالسؤال:\n{response["السؤال"]}\n\nالاجابة: \n{response["text"]}')

السياق:
<h1>الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

<h1>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

<h1>الغرامة المالية 3000 - 6000 ريال</h1>
<p>إجراء سباق للمركبات على الطرق العامة، أو السير في مواكب دون الحصول على تصريح.</p>

<h1>المراوغة بسرعة بين المركبات على الطرق العامة.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

السؤال:
ماهي الغرامة على القيادة بدون رخصة؟

الاجابة: 
لا توجد غرامة مالية مذكورة على القيادة بدون رخصة في النص الم 제공.
